In [ ]:
#import nessary libraries

from bs4 import BeautifulSoup as bs
import requests       
import smtplib        #sending emails
import time
import datetime
import csv
import pandas as pd    #making Dataframes


In [ ]:
#Sending mail for users if price is reduced 

def send_mail():
    server=smtplib.SMTP_SSL('smtp.gmail.com',465)
    server.ehlo()
    server.login('tuanminhajseedin@gmail.com','xxxxxxxxxx')

    subject='The shirt you want is below $15! Now is your chance to buy!'
    body="Tuan, This is the moment we have been waiting for, now is your chance to pick this great chance. Don't miss this chance"

    msg=f"Subject:{subject}\n\n{body}"

    server.sendmail(
        'tuanminhajseedin@gmail.com',msg
    )


#### Method 01 for load data into csv

In [ ]:
#initiate a csv file to load the scraped data
headers=['Title','Star','Price','Availability']
with open('books_Web_Scraper_Dataset.csv','w',newline='',encoding='UTF8') as f:
    writer=csv.writer(f)
    writer.writerow(headers)

In [ ]:
#connecting to the website and scrape all the data from the website and take into local
books=[]
def book_details():
    for i in range(1,51):
        url=f'https://books.toscrape.com/catalogue/page-{i}.html'
        # print(url)

        page=requests.get(url)
        soup1=bs(page.content,"html.parser")   #pulling all the html from the website
        
        #find searching under tag name of ol(order_list) and find by article with required class
        order_list=soup1.find('ol')
        articles=order_list.find_all('article',class_='product_pod')

        
        
        #retrieve all the titles inside the articles and taking the all titles in the first page
        for article in articles:        #looping through each articles
            image=article.find('img')       #find all img tags
            title=image.attrs['alt']        #filter out the title with it attributes
            star=article.find('p')          #star ratings are under p tags we can straight scrape it
            star=star['class'][1]           #to retrieve only rating value we need to get 1 index values
            price=article.find('p',class_='price_color').text     #finding the price values, those are under p tags with class of price_color and filter it's text value
            price=float(price[1:])             #text values are return with strings. we need to change it's type as float to store in csv
            availability=article.find('p',class_='instock availability').text
            availability=availability.strip()    #remove unnessary spaces 

            books.append([title,star,price,availability])
        
            data=[title,star,price,availability]

            with open('books_Web_Scraper_Dataset.csv','a+',newline='',encoding='UTF8') as f:    #appending all the data into the csv file
                writer=csv.writer(f)
                writer.writerow(data)


            if price<50:                      #sending a mail for user if price is reduced
                send_mail()


#### Method 2 for load data into csv

In [ ]:
#making Dataframes

df=pd.DataFrame(books,columns=headers)

In [ ]:
df.head()

In [ ]:
# convert dataframe into csv file format

df.to_csv('books.csv')


In [ ]:
#Automate the process in every single day and update the csv file
while True:                           
    book_details()
    time.sleep(3600)